### This notebook was used to explore the data and develop some proof of concept visualisations for use in the final app.
It's recommended to view the [rendered notebook on nbviewer](https://nbviewer.org/github/ismailmo1/mfp-wrapped/blob/main/notebooks/mfp_plotting.ipynb).

In [ ]:
import pandas as pd
import plotly.express as px
import datetime
import sys
import warnings
from plotly.offline import init_notebook_mode

warnings.filterwarnings('ignore')
init_notebook_mode(connected=True)

#hacky way to use sibling module 
# add ` "python.analysis.extraPaths": ["app/"] ` to .vscode/settings.json for pylance
sys.path.append('../app')
import myfitnesspal.analysis as mfpa
import myfitnesspal.diary_scraping as mfps

load in some myfitnesspal data using the ```get_diary_data``` function

In [ ]:
diary_df = pd.DataFrame()
for df in mfps.get_diary_data(datetime.date(2022,3,18), datetime.date(2022,3,22), user="ismailmo"):
    diary_df = pd.concat([diary_df, df], axis=0, join="outer")

convert grams to calories

In [ ]:
macro_kcals = {'carbs_g':4, 'fat_g':9, 'protein_g':4}

#add kcal column for macros
for key,val in macro_kcals.items():
    diary_df[key.split('_')[0]+'_kcal'] = diary_df[key] * val
diary_df_kcals = diary_df[['food', 'calories_kcal', 'carbs_kcal',
       'fat_kcal', 'protein_kcal', 'date']]
diary_df_kcals.head()

go from wide to long format to make plotting easier

In [ ]:
melted_df = diary_df_kcals.drop("calories_kcal", axis=1).melt(['food', 'date'])
melted_df = melted_df.rename({"value":"kcal"}, axis=1)
melted_df['meals'] = 1

melted_df = melted_df.groupby(['food', 'date', 'variable']).sum()
melted_df= melted_df.reset_index()
melted_df[melted_df['date']==datetime.date(2022,3,21)]

In [ ]:
fig = px.treemap(melted_df, path=[px.Constant("all"),
'variable','food', 'date'], values='kcal', hover_data=['meals'])

fig.show()

## summarise daily data to plot goals vs actuals

In [ ]:
diary_df['date'] = pd.to_datetime(diary_df['date'])
macro_kcals = {'carbs':4, 'fat':9, 'protein':4}
macro_colours = {'carbs':"darkturquoise", 'fat':"tomato", 'protein':"mediumorchid"}
#add kcal column for macros
for key,val in macro_kcals.items():
    diary_df[key+'_kcal'] = diary_df[key+"_g"] * val
    diary_df["goal_" + key+'_kcal'] = diary_df["goal_" +key+"_g"] * val
sum_cols = {col:'sum' for col in ['calories_kcal', 'carbs_g', 'fat_g', 'protein_g','carbs_kcal', 'fat_kcal', 'protein_kcal']}
mean_cols = {col:'mean' for col in [ 'goal_calories_kcal', 'goal_carbs_g', 'goal_fat_g',
       'goal_protein_g', 'goal_carbs_kcal', 'goal_fat_kcal',
       'goal_protein_kcal']}
agg_dict = {**sum_cols, **mean_cols}
daily_data = diary_df.groupby('date').agg(agg_dict)

daily_data.head()

In [ ]:
kcal_macro_cols = [col for col in daily_data.columns if 'kcal' in col and 'calories' not in col]
gram_macro_cols = [col for col in daily_data.columns if '_g' in col and 'calories' not in col]
calorie_cols = [col for col in daily_data.columns if 'calories' in col]

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()


for col in gram_macro_cols:
    macro = col.split("_")[-2]
    color = macro_colours[macro]
    if 'goal' in col:
        line={"dash":"dash", "color": color}
        fig.add_trace(go.Scatter(x=daily_data.index, 
        y=daily_data[col], 
        mode='lines', 
        name="goal", 
        line=line,        
        legendgroup=macro))
    else:
        fig.add_trace(go.Bar(x=daily_data.index, 
        y=daily_data[col], 
        name=macro,
        marker_color= color,
        legendgroup=macro,
        customdata =daily_data[macro+"_g"],
        hovertemplate='%{customdata}'))
fig.update_layout(title='Actual Intake vs Goals', xaxis_title ="Date", yaxis_title="Calories")
fig.show()